In [1]:
from langgraph.graph import StateGraph,START, END
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
llm = HuggingFaceEndpoint(
    repo_id="meta-llama/Llama-3.2-3B-Instruct",
    task="text-generation",
    temperature=0.1,  
    max_new_tokens=100,
)
model = ChatHuggingFace(llm=llm)

In [4]:
#Create a state

class LLMState(TypedDict):
    question: str
    answer: str

In [5]:
def llm_qa(state: LLMState) -> LLMState:

    question = state["question"]

    prompt = f"Answer the following question {question}"

    answer = model.invoke(prompt).content

    state["answer"] = answer

    return state

In [8]:
#Create Graph

graph = StateGraph(LLMState)

#Add Nodes
graph.add_node("llm_qa",llm_qa)

#Add Edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

#Compile Graph
workflow = graph.compile()

In [9]:
initial_state = {"question":"What is the capital of France?"}

final_state = workflow.invoke(initial_state)

print(final_state)

{'question': 'What is the capital of France?', 'answer': 'The capital of France is Paris.'}
